# Set Up

## Import Packages

In [ ]:
# Lavorare con i dataframe
import pandas as pd
pd.set_option('display.max_columns', None) # Per visualizzare tutte le colonne di un dataset, display(df)
#pd.set_option('display.max_rows', None) # Per visualizzare tutte le righe di un dataset, display(df)
import numpy as np

# Importare sys per indicare il path della cartella ove sono collocate le funzioni selfmade.
import sys
#sys.path.append('/Users/mattia/Desktop/MatchAnalysis_Imputation/Progetto/Funzioni')
from funzioni import alfabeto, pulisci_prezzo, pulisci_recensione, pulisci_distanza_centro, trasforma_in_data # Funzioni fatte da se


path_data_lake = "/Users/mattia/Desktop/MatchAnalysis_Imputation/Progetto/Data Lake"
path_data_product = "/Users/mattia/Desktop/MatchAnalysis_Imputation/Progetto/Data Product"

## Load the Data

Importare i dataframe contenenti informazioni su hotel e B&B riguardanti le città di Roma e Terni disponibili
sulle piattaforme di agoda e booking.     
Dati ottenuti tramite **web scraping**, per osservare il processo di scraping guardare al folder dedicato.

In [126]:
agoda_terni = pd.read_csv(f'{path_data_lake}/agoda_Terni.csv',index_col=0)
booking_terni = pd.read_csv(f'{path_data_lake}/booking_Terni.csv',index_col=0)

booking_roma = pd.read_csv(f'{path_data_lake}/booking_Roma.csv',index_col=0)
agoda_roma = pd.read_csv(f'{path_data_lake}/agoda_Roma.csv',index_col=0)

# Data overview

Osservare i dati per:
 - Capire se c'è possibilità di feature engineering;
 - Capire inoltre se ci sono delle variabili
che necessitano di pulizia.

## Dati riguardanti Agoda

Visualizzazione dati:

In [127]:
display(agoda_roma.head(1))
display(agoda_terni.head(1))

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione
0,Raeli Hotel Lazio,"7,7",Ottimo,321 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€159,"Stazione centrale Roma Termini, Roma - In pien...",3


,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione
0,Hotel Valentino,"9,0",Eccezionale,2 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",258 €,"Terni, Terni",4.0


Controllare che le variabili tra sul sito di **agoda** riguardo a Roma e Terni siano le stesse.

In [128]:
agoda_terni.columns.equals(agoda_roma.columns)

True

Le variabili tra agoda Roma e agoda Terni sono le stesse e sono:
   1. **Titolo** -> Nome della struttura;

   2. **Recensione voto numerico** -> Recensione con voto numerico;

   3. **Recensione voto parola** -> Recensione con voto a parola;

   4. **Numero recensioni** -> Numero di recensioni fatte alla struttura;

   5. **Date** -> Data di permanenza nella struttura;

   6. **Permanenza** -> Informazioni sulla permanenza, numero di notti e numero di persone (adulti o bambini);

   7. **Prezzo** -> Costo della permanenza;

   8. **Indirizzo** -> Indirizzo della struttura formato (città, provincia);

   9. **Valutazione** -> Le stelle di un hotel o la valutazione assegnata al B&B.

Sul dataset di agoda si può fare del feature engineering:

   - Creare variabile NUMERO NOTTI dalla variabile **permanenza**, deve essere un numero;

   - Creare variabile NUMERO PERSONE (adulte) e NUMERO PERSONE (bambini) dalla variabile **permanenza**;

   - Creare variabile DATA CHECK IN da **date**;

   - Creare variabile DATA CHECK OUT da **date**.



Per quanto riguarda la variabile **indirizzo**:
   - Nel dataframe **ROMA** ha il formato (zona, città - distanza centro)  es. Stazione centrale Roma Termini, Roma - In pieno centro;

   - Nel dataframe **TERNI** ha il formato (zona, città)  es. Terni, Terni;


Perciò per quanto riguarda Roma può anche essere creata la variabile **distanza dal centro**, oltre a **zona** e **città**.      
Invece riguarda Terni può essere creata la variabile **zona** e **città**.                  

Inoltre per effettuare le dovute analisi bisogna:
   - Ricodificare la variabile **prezzo** rimuovendo il simbolo dell'euro (accertarsi che i prezzi siano tutti in euro);

   - Sempre riguardo alla variabile **prezzo** eliminare il punto come separatore delle migliaia;

   - **numero recensioni** qualora comparisse "Novità su Agoda" inserire 0 (zero).

   - **numero recensioni** estrarre solo il numero di recensioni eliminado la stringa **recensioni** contenuta.

   - La variabile **distanza centro** andrà pulita e resa un numero (float).

## Dati riguardanti booking

Visualizzazione dati:

In [129]:
display(booking_roma.head(1))
display(booking_terni.head(1))

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,The Sereno-3,"8,1",Ottimo,10 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 242,"3,5 km dal centro","Trionfale, Roma",NaN,4.0,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...


,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,Hotel De Paris,"7,2",Buono,869 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"0,9 km dal centro",Terni,3.0,NaN,Camera Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Doppia con Letti Singoli2 letti singoli...


Controllare che le variabili tra sul sito di **booking** riguardo a Roma e Terni siano le stesse.

In [130]:
booking_terni.columns.equals(booking_roma.columns)

True

Le variabili tra agoda Roma e agoda Terni sono le stesse e sono:
   1. **Titolo** -> Nome della struttura;

   2. **Recensione voto numerico** -> Recensione con voto numerico;

   3. **Recensione voto parola** -> Recensione con voto a parola;

   4. **Numero recensioni** -> Numero di recensioni fatte alla struttura;

   5. **Date** -> Data di permanenza nella struttura;

   6. **Permanenza** -> Informazioni sulla permanenza, numero di notti e numero di personev(adulti o bambini);

   7. **Prezzo** -> Costo della permanenza;

   8. **Distanza centro** -> Distanza dal centro della città di riferimento; 

   9. **Indirizzo** -> Indirizzo della struttura formato (città, provincia);

   10. **Stelle** -> Stelle se la struttura è un hotel (NAN se non è hotel);

   11. **Valutazione booking** -> Valutazione assegnata al B&B (NAN se non è B&B);

   12. **Descrizione camera** -> Descrizione della camera, quante camere se c'è vista ecc...;

   13. **Configurazione camera** -> Se c'è il bagno, numero di letti ecc...;

   14. **Descrizione unprocessed** -> Descrizione camera + Configurazione camera.

Sul dataset di booking si può fare del feature engineering:
   - Creare variabile NUMERO NOTTI dalla variabile **permanenza**, deve essere un numero;

   - Creare variabile NUMERO PERSONE (adulte) e NUMERO PERSONE (bambini) dalla variabile **permanenza**;

   - Creare variabile DATA CHECK IN da **date**;

   - Creare variabile DATA CHECK OUT da **date**.

Per quanto riguarda la variabile **indirizzo**:
   - Nel dataframe **ROMA** ha il formato (zona, città)  es. Trionfale, Roma;

   - Nel dataframe **TERNI** ha il formato (città)  es. Terni;


Perciò per quanto riguarda Roma può anche essere creata la variabile **distanza dal centro**, oltre a **zona** e **città**.      
Invece riguarda Terni può essere creata la variabile **zona** e **città**.   

Inoltre per effettuare le dovute analisi bisogna:
   - Ricodificare la variabile **prezzo** rimuovendo il simbolo dell'euro (accertarsi che i prezzi siano tutti in euro);

   - Sempre riguardo alla variabile **prezzo** eliminare il punto come separatore delle migliaia;

   - **numero recensioni** qualora comparisse "Novità su booking" inserire 0 (zero).

   - **numero recensioni** estrarre solo il numero di recensioni eliminado la stringa **recensioni** contenuta.

   - La variabile **distanza centro** andrà pulita e resa un numero (float).

# Feature engineering

Creazione di nuove variabili partendo da quelle che si ha per estrarre informazioni aggiuntive.

## Feature engineering applicato ad Agoda

Queste trasformazioni saranno le stesse applicate al dataframe **agoda roma** e **agoda terni**. Si lavora su:
   - Numero notti;
   - Numero persone -> Numero adulti, numero bambini, totale ospiti;
   - Inizio permanenza;
   - Fine permanenza.
   
Per quanto riguarda le variabili da estrarre da **indirizzo** queste andranno trattate diversamente tra agoda ROMA e agoda TERNI:
   - Distanza centro;
   - Zona;
   - Città.



Già da questa fase di feature engineering si potrebbero unire i dataset riguardanti lo stesso sito web, per evitare errori sarà fatto nella fase di cleaning.

In [131]:
#pd.set_option('display.max_rows', 10)
agoda_terni.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione
0,Hotel Valentino,"9,0",Eccezionale,2 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",258 €,"Terni, Terni",4.0


### Agoda Terni

In [132]:
# Numero notti, estraggo il numero di notti dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita.
agoda_terni["numero_notti"]=agoda_terni.permanenza.str.split(",").str[0]

# Numero persone, estraggo il numero di persone dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà ancora più elaborata e verranno create le variabili. Tuttavia  numero di adulti, Numero bambini, Numero totale ospiti non vengono create perchè ci sono solo 2 adulti per tutto il dataset.
agoda_terni["numero_persone"]=agoda_terni.permanenza.str.split(",").str[1]

# Data di inizio permanenza, estraggo la data di inzio permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
agoda_terni["inizio_permanenza"]=agoda_terni.date.str.split("-").str[0]

# Data di fine permanenza, estraggo la data di fine permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
agoda_terni["fine_permanenza"]=agoda_terni.date.str.split("-").str[1]

#-----#
# Variabile da estrarre da indirizzo.

# Distanza centro, estratto dalla variabile indirizzo. Non è presente la distanza dal centro.
agoda_terni["distanza_centro"] = np.nan

# Zona, estratta dalla variabile indirizzo.
agoda_terni["zona"] = agoda_terni.indirizzo.str.split(",").str[0]

# Città, estratta dalla variabile indirizzo.
agoda_terni["città"] = agoda_terni.indirizzo.str.split(",").str[1]

In [133]:
agoda_terni.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,distanza_centro,zona,città
0,Hotel Valentino,"9,0",Eccezionale,2 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",258 €,"Terni, Terni",4.0,1 notte,2 adulti,1 agosto,2 agosto,NaN,Terni,Terni


### Agoda Roma

In [134]:
# Numero notti, estraggo il numero di notti dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita.
agoda_roma["numero_notti"]=agoda_roma.permanenza.str.split(",").str[0]

# Numero persone, estraggo il numero di persone dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà ancora più elaborata e verranno create le variabili. Tuttavia  numero di adulti, Numero bambini, Numero totale ospiti non vengono create perchè ci sono solo 2 adulti per tutto il dataset.
agoda_roma["numero_persone"]=agoda_roma.permanenza.str.split(",").str[1]

# Data di inizio permanenza, estraggo la data di inzio permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
agoda_roma["inizio_permanenza"]=agoda_roma.date.str.split("-").str[0]

# Data di fine permanenza, estraggo la data di fine permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
agoda_roma["fine_permanenza"]=agoda_roma.date.str.split("-").str[1]

#-----#
# Variabile da estrarre da indirizzo.

# Distanza centro, estratto dalla variabile indirizzo.
agoda_roma["distanza_centro"] = agoda_roma.indirizzo.str.split("-").str[1]

# Zona, estratta dalla variabile indirizzo.
agoda_roma["zona"] = agoda_roma.indirizzo.str.split("-").str[0].str.split(",").str[0]

# Città, estratta dalla variabile indirizzo.
agoda_roma["città"] = agoda_roma.indirizzo.str.split("-").str[0].str.split(",").str[1]

In [135]:
agoda_roma.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,distanza_centro,zona,città
0,Raeli Hotel Lazio,"7,7",Ottimo,321 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€159,"Stazione centrale Roma Termini, Roma - In pien...",3,1 notte,2 adulti,1 agosto,2 agosto,In pieno centro,Stazione centrale Roma Termini,Roma


## Feature engineering applicato a Booking

### Booking Terni

In [136]:
booking_terni.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,Hotel De Paris,"7,2",Buono,869 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"0,9 km dal centro",Terni,3.0,NaN,Camera Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Doppia con Letti Singoli2 letti singoli...


In [137]:
# Numero notti, estraggo il numero di notti dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita.
booking_terni["numero_notti"]=booking_terni.permanenza.str.split(",").str[0]

# Numero persone, estraggo il numero di persone dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà ancora più elaborata e verranno create le variabili. Tuttavia  numero di adulti, Numero bambini, Numero totale ospiti non vengono create perchè ci sono solo 2 adulti per tutto il dataset.
booking_terni["numero_persone"]=booking_terni.permanenza.str.split(",").str[1]

# Data di inizio permanenza, estraggo la data di inzio permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
booking_terni["inizio_permanenza"]=booking_terni.date.str.split("-").str[0]

# Data di fine permanenza, estraggo la data di fine permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
booking_terni["fine_permanenza"]=booking_terni.date.str.split("-").str[1]

# Zona, estratta dalla variabile indirizzo.
booking_terni["zona"] = booking_terni.indirizzo

# Città, estratta dalla variabile indirizzo.
booking_terni["città"] = booking_terni.indirizzo

In [138]:
booking_terni.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,zona,città
0,Hotel De Paris,"7,2",Buono,869 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"0,9 km dal centro",Terni,3.0,NaN,Camera Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Doppia con Letti Singoli2 letti singoli...,1 notte,2 adulti,1 agosto,2 agosto,Terni,Terni


### Booking Roma

In [139]:
booking_roma.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,The Sereno-3,"8,1",Ottimo,10 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 242,"3,5 km dal centro","Trionfale, Roma",NaN,4.0,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...


In [140]:
# Numero notti, estraggo il numero di notti dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita.
booking_roma["numero_notti"]=booking_roma.permanenza.str.split(",").str[0]

# Numero persone, estraggo il numero di persone dalla variabile permanenza e lo assegno ad una nuova variabile. Successivamente la variabile sarà ancora più elaborata e verranno create le variabili. Tuttavia  numero di adulti, Numero bambini, Numero totale ospiti non vengono create perchè ci sono solo 2 adulti per tutto il dataset.
booking_roma["numero_persone"]=booking_roma.permanenza.str.split(",").str[1]

# Data di inizio permanenza, estraggo la data di inzio permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
booking_roma["inizio_permanenza"]=booking_roma.date.str.split("-").str[0]

# Data di fine permanenza, estraggo la data di fine permanenza  dalla variabile 'date' e lo assegno ad una nuova variabile. Successivamente la variabile sarà pulita ulteriormente.
booking_roma["fine_permanenza"]=booking_roma.date.str.split("-").str[1]

# Zona, estratta dalla variabile indirizzo.
booking_roma["zona"] = booking_roma.indirizzo.str.split(",").str[0]

# Città, estratta dalla variabile indirizzo.
booking_roma["città"] = booking_roma.indirizzo.str.split(",").str[1]

In [141]:
booking_roma.head(1)

,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,zona,città
0,The Sereno-3,"8,1",Ottimo,10 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 242,"3,5 km dal centro","Trionfale, Roma",NaN,4.0,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...,1 notte,2 adulti,1 agosto,2 agosto,Trionfale,Roma


# Data Cleaning

## Riodinare le variabili

Solo per maggiore ordine e chiarezza riordino le variabili.

In [142]:
agoda_roma = agoda_roma[['titolo','zona','città','distanza_centro','prezzo','numero_notti','numero_persone','inizio_permanenza',
'fine_permanenza', 'recensione_voto_numerico', 'recensione_voto_parola',
'numero_recensioni', 'date', 'permanenza', 'indirizzo',
'valutazione']]

agoda_terni = agoda_terni[['titolo','zona','città','distanza_centro','prezzo','numero_notti','numero_persone','inizio_permanenza',
'fine_permanenza', 'recensione_voto_numerico', 'recensione_voto_parola',
'numero_recensioni', 'date', 'permanenza', 'indirizzo',
'valutazione']]

booking_roma = booking_roma[['titolo','zona','città','distanza_centro','prezzo','numero_notti','numero_persone','inizio_permanenza',
'fine_permanenza', 'recensione_voto_numerico', 'recensione_voto_parola',
'numero_recensioni', 'date', 'permanenza', 'indirizzo','valutazione_booking','stelle',  'descrizione_camera',
       'configurazione_camera', 'descrizione_unprocessed']]

booking_terni = booking_terni[['titolo','zona','città','distanza_centro','prezzo','numero_notti','numero_persone','inizio_permanenza',
'fine_permanenza', 'recensione_voto_numerico', 'recensione_voto_parola',
'numero_recensioni', 'date', 'permanenza', 'indirizzo','valutazione_booking','stelle',  'descrizione_camera',
       'configurazione_camera', 'descrizione_unprocessed']]

## Cleaning dei dati riguardanti Agoda

Come prima cosa si uniscono i datataset **agoda_roma** e **agoda_terni** così da lavorare su un solo dataset.

In [143]:
agoda = pd.concat([agoda_roma,agoda_terni]) # Unisco agoda_roma e agoda_terni
agoda.reset_index(inplace=True,drop=True) # Nuovi indici, Eliminare quelli precedenti

In [144]:
display(agoda.head(1))
display(agoda.tail(1))

,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione
0,Raeli Hotel Lazio,Stazione centrale Roma Termini,Roma,In pieno centro,€159,1 notte,2 adulti,1 agosto,2 agosto,"7,7",Ottimo,321 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0


,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione
676,Casa vacanze TERRA DI ASLAN,Narni,Narni,NaN,119 €,1 notte,2 adulti,1 agosto,2 agosto,"9,6",Eccezionale,7 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti","Narni, Narni",3.0


In [145]:
# Pulizia della variabile 'prezzo' : rimosso simbolo euro ed il punto separatore delle migliaia tramite la funzione pulisci_prezzo, variabile trasformata da object a int.
agoda.prezzo = pulisci_prezzo(agoda.prezzo)

# Pulizia della variabile 'numero di recensioni' -> rimuove la parola 'recensioni, Se la recensione risulta 'novità su agoda' assegna 0. rimuove anche il separatore delle migliaia. Discutible l'assegnazione di 0 recensioni quando è una novita.
agoda.numero_recensioni =  agoda.numero_recensioni.apply(lambda row: pulisci_recensione(row))

# Pulizia della variabile 'distanza_centro' -> Lascia solo la distanza in km.
agoda["distanza_centro"] =  agoda.distanza_centro.apply(lambda row: pulisci_distanza_centro(row))

# Rendere la variabile numero di persone un intero ed eliminare la parola 'adulti'
agoda["numero_persone"]=agoda.numero_persone.str.replace("adulti","").astype(int)

# Rendere la variabile numero di notti un intero ed eliminare la parola 'notte'
agoda["numero_notti"]=agoda.numero_notti.str.replace("notte","").astype(int)

# Pulizia della variabile 'inizio_permanenza' -> Trasforma la data in formato italiano.
agoda["inizio_permanenza"] =  agoda.inizio_permanenza.apply(lambda row: trasforma_in_data(row))

# Trasformare la colonna in un datetime (formato datetime, standard python)
agoda["inizio_permanenza_datetime"]=pd.to_datetime(agoda["inizio_permanenza"])

# Pulizia della variabile 'fine_permanenza' -> -> Trasforma la data in formato italiano.
agoda["fine_permanenza"] =  agoda.fine_permanenza.apply(lambda row: trasforma_in_data(row))

# Trasformare la colonna in un datetime (formato datetime, standard python)
agoda["fine_permanenza_datetime"]=pd.to_datetime(agoda["fine_permanenza"])


In [146]:
display(agoda.head(1))
display(agoda.tail(1))

,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime
0,Raeli Hotel Lazio,Stazione centrale Roma Termini,Roma,0.0,159,1,2,01-08-25,02-08-25,"7,7",Ottimo,321,1 agosto - 2 agosto,"1 notte, 2 adulti","Stazione centrale Roma Termini, Roma - In pien...",3.0,2025-01-08,2025-02-08


,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione,inizio_permanenza_datetime,fine_permanenza_datetime
676,Casa vacanze TERRA DI ASLAN,Narni,Narni,NaN,119,1,2,01-08-25,02-08-25,"9,6",Eccezionale,7,1 agosto - 2 agosto,"1 notte, 2 adulti","Narni, Narni",3.0,2025-01-08,2025-02-08


In [147]:
agoda.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   titolo                      677 non-null    object        
 1   zona                        677 non-null    object        
 2   città                       677 non-null    object        
 3   distanza_centro             539 non-null    float64       
 4   prezzo                      677 non-null    int64         
 5   numero_notti                677 non-null    int64         
 6   numero_persone              677 non-null    int64         
 7   inizio_permanenza           677 non-null    object        
 8   fine_permanenza             677 non-null    object        
 9   recensione_voto_numerico    677 non-null    object        
 10  recensione_voto_parola      677 non-null    object        
 11  numero_recensioni           677 non-null    int64         

## Cleaning dei dati riguardanti Booking

Come prima cosa si uniscono i datataset **booking_roma** e **booking_terni** così da lavorare su un solo dataset.

In [148]:
booking = pd.concat([booking_roma,booking_terni]) # Unisco agoda_roma e agoda_terni
booking.reset_index(inplace=True,drop=True) # Nuovi indici, Eliminare quelli precedenti
display(booking.head(1))
display(booking.tail(1))

,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed
0,The Sereno-3,Trionfale,Roma,"3,5 km dal centro",€ 242,1 notte,2 adulti,1 agosto,2 agosto,"8,1",Ottimo,10 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti","Trionfale, Roma",4.0,NaN,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...


,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed
885,-15min Marmore- Ampio Appartamento Terra e Cuore,Terni,Terni,"2,9 km dal centro",€ 182,1 notte,2 adulti,1 agosto,2 agosto,Novità su booking,Novità su booking,Novità su booking,1 agosto - 2 agosto,"1 notte, 2 adulti",Terni,NaN,NaN,Appartamento con 2 Camere da Letto,Configurazione camera non disponibile,Appartamento con 2 Camere da Letto4 letti (2 s...


In [ ]:
# Pulizia della variabile 'prezzo' : rimosso simbolo euro ed il punto separatore delle migliaia tramite la funzione pulisci_prezzo, variabile trasformata da object a int.
booking.prezzo = pulisci_prezzo(booking.prezzo)

# Pulizia della variabile 'numero di recensioni' -> rimuove la parola 'recensioni, Se la recensione risulta 'novità su agoda' assegna 0. rimuove anche il separatore delle migliaia. Discutible l'assegnazione di 0 recensioni quando è una novita.
booking.numero_recensioni =  booking.numero_recensioni.apply(lambda row: pulisci_recensione(row))

# Pulizia della variabile 'distanza_centro' -> Lascia solo la distanza in km.
booking["distanza_centro"] =  booking.distanza_centro.apply(lambda row: pulisci_distanza_centro(row))

# Pulizia variabili: recensione_voto_parola, recensione_voto_parola mettere NAN dove è specificato 'Novità su booking'
booking.loc[booking.recensione_voto_numerico == "Novità su booking","recensione_voto_numerico"] = np.nan # Questo è il formato giusto per assegnare un nuovo valore ad una variabile nel dataframe che rispetta alcune condizioni
booking.loc[booking.recensione_voto_parola == "Novità su booking","recensione_voto_parola"] = np.nan

# Rendere la variabile numero di persone un intero ed eliminare la parola 'adulti'
booking["numero_persone"]=booking.numero_persone.str.replace("adulti","").astype(int)

# Rendere la variabile numero di notti un intero ed eliminare la parola 'notte'
booking["numero_notti"]=booking.numero_notti.str.replace("notte","").astype(int)

# Pulizia della variabile 'inizio_permanenza' -> Trasforma la data in formato italiano.
booking["inizio_permanenza"] =  booking.inizio_permanenza.apply(lambda row: trasforma_in_data(row))

# Trasformare la colonna in un datetime (formato datetime, standard python)
booking["inizio_permanenza_datetime"]=pd.to_datetime(booking["inizio_permanenza"])

# Pulizia della variabile 'fine_permanenza' -> -> Trasforma la data in formato italiano.
booking["fine_permanenza"] =  booking.fine_permanenza.apply(lambda row: trasforma_in_data(row))

# Trasformare la colonna in un datetime (formato datetime, standard python)
booking["fine_permanenza_datetime"]=pd.to_datetime(booking["fine_permanenza"])

In [150]:
display(booking.head(1))
display(booking.tail(1))

,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed,inizio_permanenza_datetime,fine_permanenza_datetime
0,The Sereno-3,Trionfale,Roma,3.5,242,1,2,01-08-25,02-08-25,"8,1",Ottimo,10,1 agosto - 2 agosto,"1 notte, 2 adulti","Trionfale, Roma",4.0,NaN,Appartamento con 2 Camere da Letto e Vista Città,Intero appartamento • 2 camere da letto • 1 ba...,Appartamento con 2 Camere da Letto e Vista Cit...,2025-01-08,2025-02-08


,titolo,zona,città,distanza_centro,prezzo,numero_notti,numero_persone,inizio_permanenza,fine_permanenza,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,indirizzo,valutazione_booking,stelle,descrizione_camera,configurazione_camera,descrizione_unprocessed,inizio_permanenza_datetime,fine_permanenza_datetime
885,-15min Marmore- Ampio Appartamento Terra e Cuore,Terni,Terni,2.9,182,1,2,01-08-25,02-08-25,NaN,NaN,0,1 agosto - 2 agosto,"1 notte, 2 adulti",Terni,NaN,NaN,Appartamento con 2 Camere da Letto,Configurazione camera non disponibile,Appartamento con 2 Camere da Letto4 letti (2 s...,2025-01-08,2025-02-08


In [151]:
booking.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 886 entries, 0 to 885
Data columns (total 22 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   titolo                      886 non-null    object        
 1   zona                        886 non-null    object        
 2   città                       861 non-null    object        
 3   distanza_centro             886 non-null    float64       
 4   prezzo                      886 non-null    int64         
 5   numero_notti                886 non-null    int64         
 6   numero_persone              886 non-null    int64         
 7   inizio_permanenza           886 non-null    object        
 8   fine_permanenza             886 non-null    object        
 9   recensione_voto_numerico    881 non-null    object        
 10  recensione_voto_parola      881 non-null    object        
 11  numero_recensioni           886 non-null    int64         

# ROBA SOF

### Agoda Roma

In [ ]:
# Pulizia della variabile 'prezzo' : rimosso simbolo euro ed il punto separatore delle migliaia tramite la funzione pulisci_prezzo, variabile trasformata da object a int.
agoda_roma.prezzo = pulisci_prezzo(agoda_roma.prezzo)
agoda_roma

In [4]:
# Aggiungere variabili e modificarle (feature engeneering)
## Estrarre solo il primo nome, assegnarlo a variabile 'città'
agoda_terni['città'] = agoda_terni['indirizzo'].str.split(',').str[0].str.strip()
agoda_terni['comune'] = agoda_terni['indirizzo'].str.split(',').str[0].str.strip()

## Creare nuova variabile città, utile in seguito
booking_terni['città'] = booking_terni['indirizzo']

## Rinominare città Papigno come Terni
agoda_terni.loc[agoda_terni['città'] == 'Papigno', 'città'] = 'Terni'

## Creare variabile città
booking_roma['città']='Roma'
agoda_roma['città']='Roma'

# Facciamo merge dei due dataset separatamente per agoda e booking
## Unione dei dataset Agoda
agoda = pd.concat([agoda_terni, agoda_roma], ignore_index=True)

## Unione dei dataset Booking
booking = pd.concat([booking_terni, booking_roma], ignore_index=True)



For the purposes of our analysis, we only want to know a title. Let's toss the other columns so we have something more manageable to work with.

In [5]:
agoda["titolo"]=agoda.titolo.apply(lambda x: alfabeto(x))

booking["titolo"]=booking.titolo.apply(lambda x: alfabeto(x))

In [6]:
# Take a first look at the data
agoda.head()

,Unnamed: 0,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,indirizzo,valutazione,città,comune
0,0,hotel valentino,"9,0",Eccezionale,2 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",258 €,"Terni, Terni",4.0,Terni,Terni
1,1,garden hotel terni,"7,9",Ottimo,505 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",138 €,"Terni, Terni",4.0,Terni,Terni
2,2,hotel michelangelo palace spa,"8,0",Fantastico,718 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",174 €,"Terni, Terni",4.0,Terni,Terni
3,3,classic hotel tulipano,"7,6",Ottimo,453 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",98 €,"Terni, Terni",3.0,Terni,Terni
4,4,de hotel paris,"8,1",Fantastico,11 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",69 €,"Terni, Terni",3.0,Terni,Terni


In [7]:
booking.head()

,Unnamed: 0,titolo,recensione_voto_numerico,recensione_voto_parola,numero_recensioni,date,permanenza,prezzo,distanza_centro,indirizzo,stelle,valutazione_booking,descrizione_camera,configurazione_camera,descrizione_unprocessed,città
0,0,de hotel paris,"7,2",Buono,869 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"0,9 km dal centro",Terni,3.0,NaN,Camera Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Doppia con Letti Singoli2 letti singoli...,Terni
1,1,bb cospea,"9,0",Eccellente,674 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 55,"1,7 km dal centro",Terni,NaN,3.0,Camera Doppia con Letti Singoli,Configurazione camera non disponibile,Camera Doppia con Letti Singoli2 letti singoli...,Terni
2,2,bedbreakfast rosy,"9,0",Eccellente,330 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 77,1 km dal centro,Terni,NaN,4.0,Camera Matrimoniale con Bagno Privato Esterno,Configurazione camera non disponibile,Camera Matrimoniale con Bagno Privato Esterno ...,Terni
3,3,angolo bb verde,"8,6",Favoloso,219 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 69,"2,6 km dal centro",Terni,NaN,3.0,Camera Matrimoniale,Configurazione camera non disponibile,Camera Matrimoniale1 letto matrimoniale alla f...,Terni
4,4,filomena guest house santa,"9,7",Eccezionale,11 recensioni,1 agosto - 2 agosto,"1 notte, 2 adulti",€ 74,1 km dal centro,Terni,NaN,3.0,Camera Tripla,Configurazione camera non disponibile,"Camera Tripla2 letti (1 singolo, 1 matrimonial...",Terni


In [8]:
# List of columns to keep
selected_columns = ['titolo','prezzo', 'indirizzo','città','numero_recensioni','valutazione_booking']

# Subset data frame to only include those columns
booking_reduced = booking[selected_columns]


In [9]:
# Look at the updated dataframe
booking_reduced.head()

,titolo,prezzo,indirizzo,città,numero_recensioni,valutazione_booking
0,de hotel paris,€ 69,Terni,Terni,869 recensioni,NaN
1,bb cospea,€ 55,Terni,Terni,674 recensioni,3.0
2,bedbreakfast rosy,€ 77,Terni,Terni,330 recensioni,4.0
3,angolo bb verde,€ 69,Terni,Terni,219 recensioni,3.0
4,filomena guest house santa,€ 74,Terni,Terni,11 recensioni,3.0


In [10]:
selected_columns = ['titolo','prezzo', 'indirizzo','città','numero_recensioni','valutazione']
agoda_reduced = agoda[selected_columns]
agoda_reduced.head()

,titolo,prezzo,indirizzo,città,numero_recensioni,valutazione
0,hotel valentino,258 €,"Terni, Terni",Terni,2 recensioni,4.0
1,garden hotel terni,138 €,"Terni, Terni",Terni,505 recensioni,4.0
2,hotel michelangelo palace spa,174 €,"Terni, Terni",Terni,718 recensioni,4.0
3,classic hotel tulipano,98 €,"Terni, Terni",Terni,453 recensioni,3.0
4,de hotel paris,69 €,"Terni, Terni",Terni,11 recensioni,3.0


# pulizia variabile indirizzo

In [11]:
#nuova pulizia indirizzo
agoda_reduced['indirizzo'] = (
    agoda_reduced['indirizzo']
    .str.lower()  # tutto minuscolo
    .str.replace(r'\s*-\s*.*$', '', regex=True)  # rimuove tutto dopo il trattino incluso
    .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
    .str.replace(r'\bcentrale\b', '', regex=True)  # rimuove "centrale" come parola intera
    .str.replace(r'\b(rione|piazza|fontana|di|del|e|città|citta|citt)\b', '', regex=True)  # rimuove parole specificate
    .str.replace(r'\s+', ' ', regex=True)  # rimuove spazi multipli eventualmente lasciati
    .str.strip()  # rimuove spazi iniziali/finali
)


/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/3028220868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agoda_reduced['indirizzo'] = (


In [12]:
agoda_reduced.indirizzo.unique()

array(['terni, terni', 'san gemini, san gemini', 'narni, narni',
       'stroncone, stroncone', 'montefranco, montefranco',
       "calvi dell' umbria, terni", 'papigno, papigno',
       'stazione termini', 'via veneto', 'trionfale', 'appio latino',
       'pantheon', 'eur garbatella', 'aurelio monteverde',
       'aeroporto fiumicino', 'ovest', 'nord', 'est', 'trastevere',
       'trevi', 'vaticano', 'flaminio parioli', 'boccea', 'nomentano',
       'navona', 'monti', 'colosseo foro romano', 'policlinico',
       'aeroporto ciampino', 'ostia', 'spagna', 'san giovanni'],
      dtype=object)

In [13]:
booking_reduced['indirizzo'] = (
    booking_reduced['indirizzo']
    .str.lower()  # tutto minuscolo
    .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
    .str.replace(r'\bcentrale\b', '', regex=True)  # rimuove "centrale" come parola intera
    .str.replace(r'\b(rione|piazza|fontana|di|del|e|città|citta|citt)\b', '', regex=True)  # rimuove parole specificate
    .str.replace(r'\s+', ' ', regex=True)  # rimuove spazi multipli eventualmente lasciati
    .str.strip())

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/542438600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_reduced['indirizzo'] = (


In [14]:
booking_reduced.indirizzo.unique()

array(['terni', 'trionfale', 'pantheon', 'villa borghese/parioli',
       'stazione termini', 'gianicolense', 'monti', '', 'centro',
       'san giovanni', 'navona', 'vaticano prati', 'trevi', 'via veneto',
       'spagna', 'trastevere', 'colosseo', 'aventino', 'tor quinto',
       'ostiense', 'eur', 'nomentano', 'magliana vecchia',
       'giuliano-dalmata', 'fonte ostiense', 'aurelio', 'appio latino',
       'portuense', 'torrino', 'monte sacro', 'san lorenzo', 'tiburtino',
       'ardeatino'], dtype=object)

In [15]:
agoda_reduced.head()

,titolo,prezzo,indirizzo,città,numero_recensioni,valutazione
0,hotel valentino,258 €,"terni, terni",Terni,2 recensioni,4.0
1,garden hotel terni,138 €,"terni, terni",Terni,505 recensioni,4.0
2,hotel michelangelo palace spa,174 €,"terni, terni",Terni,718 recensioni,4.0
3,classic hotel tulipano,98 €,"terni, terni",Terni,453 recensioni,3.0
4,de hotel paris,69 €,"terni, terni",Terni,11 recensioni,3.0


In [16]:
booking_reduced.head()

,titolo,prezzo,indirizzo,città,numero_recensioni,valutazione_booking
0,de hotel paris,€ 69,terni,Terni,869 recensioni,NaN
1,bb cospea,€ 55,terni,Terni,674 recensioni,3.0
2,bedbreakfast rosy,€ 77,terni,Terni,330 recensioni,4.0
3,angolo bb verde,€ 69,terni,Terni,219 recensioni,3.0
4,filomena guest house santa,€ 74,terni,Terni,11 recensioni,3.0


Great! Now we have what we need to do our analysis, and not a whole lot of extraneous information. Let's move on to preprocessing.

# pulizia titolo da parole roma e rome

In [17]:
#nuova pulizia indirizzo
agoda_reduced['titolo'] = (
    agoda_reduced['titolo']
    .str.lower()  # tutto minuscolo
    .str.replace(r'\s*-\s*.*$', '', regex=True)  # rimuove tutto dopo il trattino incluso
    .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
    .str.replace(r',?\s*rome\b', '', regex=True)  # rimuove "centrale" come parola intera
    .str.strip()  # rimuove spazi iniziali/finali
)

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/2607220419.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agoda_reduced['titolo'] = (


In [18]:
#nuova pulizia indirizzo
booking_reduced['titolo'] = (
    booking_reduced['titolo']
    .str.lower()  # tutto minuscolo
    .str.replace(r'\s*-\s*.*$', '', regex=True)  # rimuove tutto dopo il trattino incluso
    .str.replace(r',?\s*roma\b', '', regex=True)  # rimuove "roma" opzionalmente preceduta da virgola
    .str.replace(r',?\s*rome\b', '', regex=True)  # rimuove "centrale" come parola intera
    .str.strip()  # rimuove spazi iniziali/finali
)

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/600013662.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_reduced['titolo'] = (


# pulizia prezzo

In [19]:
# Applica la pulizia
booking_reduced['prezzo'] = pulisci_prezzo(booking_reduced['prezzo'])
agoda_reduced['prezzo'] = pulisci_prezzo(agoda_reduced['prezzo'])

/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/2615639504.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_reduced['prezzo'] = pulisci_prezzo(booking_reduced['prezzo'])
/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/2615639504.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agoda_reduced['prezzo'] = pulisci_prezzo(agoda_reduced['prezzo'])


# pulizia numero recensioni

In [20]:
# su booking
booking_reduced['numero_recensioni'] = booking_reduced['numero_recensioni']\
    .astype(str)\
    .str.extract(r'(\d+)')\
    .astype(float)

# su agoda
agoda_reduced['numero_recensioni'] = agoda_reduced['numero_recensioni']\
    .astype(str)\
    .str.extract(r'(\d+)')\
    .astype(float)


/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/4137780272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  booking_reduced['numero_recensioni'] = booking_reduced['numero_recensioni']\
/var/folders/j4/9fk3st2d30g22vff83pj6cnw0000gn/T/ipykernel_8489/4137780272.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  agoda_reduced['numero_recensioni'] = agoda_reduced['numero_recensioni']\


In [21]:
booking_reduced.head()

,titolo,prezzo,indirizzo,città,numero_recensioni,valutazione_booking
0,de hotel paris,69,terni,Terni,869.0,NaN
1,bb cospea,55,terni,Terni,674.0,3.0
2,bedbreakfast rosy,77,terni,Terni,330.0,4.0
3,angolo bb verde,69,terni,Terni,219.0,3.0
4,filomena guest house santa,74,terni,Terni,11.0,3.0


<a id='Preprocess:-clean-Up-Names-and-separate-to-first-middle-and-last-name'></a>
## Preprocessing the Data
-----

# rinominiamo i dataset

In [22]:
booking_reduced = booking_reduced.rename(columns={'titolo': 'titolo_booking', 'prezzo': 'prezzo_booking', 'indirizzo': 'indirizzo_booking','città':'città_booking','numero_recensioni':'numero_recensioni_booking'})

In [23]:
agoda_reduced = agoda_reduced.rename(columns={'titolo': 'titolo_agoda', 'prezzo': 'prezzo_agoda', 'indirizzo': 'indirizzo_agoda','città':'città_agoda','numero_recensioni':'numero_recensioni_agoda','valutazione':'valutazione_agoda'})

In [24]:
booking_reduced.shape

(886, 6)

# AGGIUNTA FAKE RECORD

-----------------------------------------------------------------

hotel del valentino ma che sta a Roma

In [25]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_agoda': 'hotel valentino', 'prezzo_agoda': 190 ,'indirizzo_agoda':'eur','città_agoda':'Roma'}])

# Aggiunta al dataset
agoda_reduced = pd.concat([agoda_reduced, nuova_riga], ignore_index=True)

In [26]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_agoda': 'paris hotel', 'prezzo_agoda': 110 ,'indirizzo_agoda':'termini','città_agoda':'Roma'}])

# Aggiunta al dataset
agoda_reduced = pd.concat([agoda_reduced, nuova_riga], ignore_index=True)

In [27]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_agoda': 'fiori hotel', 'prezzo_agoda': 81 ,'indirizzo_agoda':'Terni','città_agoda':'Terni'}])

# Aggiunta al dataset
agoda_reduced = pd.concat([agoda_reduced, nuova_riga], ignore_index=True)

inverto ordine parole

(non funziona perchè fa index sulle lettere) cambiare con matching var

errore battitura
classic hotel Tulipano e classic hotel Tulippano sono matchati correttamente

In [28]:
# Nuova riga da aggiungere
nuova_riga = pd.DataFrame([{'titolo_agoda': 'classic hotel tulippano', 'prezzo_agoda': 109 ,'indirizzo_agoda':'Roma','città_agoda':'Roma'}])

# Aggiunta al dataset
agoda_reduced = pd.concat([agoda_reduced, nuova_riga], ignore_index=True)

-----------------------------------------------------------------

# Esportazione dati

In [29]:
agoda_reduced.to_csv(f"{path_data_product}/agoda.csv")

In [30]:
booking_reduced.to_csv(f"{path_data_product}/booking.csv")